# Step 1 - Connect to the `PLANAPI` DB and find the rawnav file names

In [1]:
import cx_Oracle
import os, sys, os.path
import pandas as pd
from dotenv import load_dotenv
path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
path_processed_data = os.path.join(path_working, "data", "02-processed")
load_dotenv(os.path.join(path_working, '.env'))
os.environ['PATH'] = 'C:\\Users\\E048374\\Anaconda3\\envs\\ox;C:\\Users\\E048374\\Anaconda3\\envs\\ox\\Library\\mingw-w64\\bin;C:\\Users\\E048374\\Anaconda3\\envs\\ox\\Library\\usr\\bin;C:\\Users\\E048374\\Anaconda3\\envs\\ox\\Library\\bin;C:\\Users\\E048374\\Anaconda3\\envs\\ox\\Scripts;C:\\Users\\E048374\\Anaconda3\\envs\\ox\\bin;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3\\Library\\mingw-w64\\bin;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3\\Library\\usr\\bin;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3\\Library\\bin;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3\\Scripts;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3\\bin;C:\\;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\Users\\E048374\\Anaconda3\\Scripts\\condabin;C:\\Users\\E048374\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\mingw64\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\usr\\local\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\usr\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\usr\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\mingw64\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\usr\\bin;C:\\Users\\E048374\\bin;C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath;C:\\oracle\\instantclient_19_6;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0;C:\\WINDOWS\\System32\\OpenSSH;C:\\Program Files\\Docker\\Docker\\resources\\bin;C:\\ProgramData\\DockerDesktop\\version-bin;C:\\Users\\E048374\\Anaconda3;C:\\Users\\E048374\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\E048374\\Anaconda3\\Library\\usr\\bin;C:\\Users\\E048374\\Anaconda3\\Library\\bin;C:\\Users\\E048374\\Anaconda3\\Scripts;C:\\Users\\E048374\\AppData\\Local\\Microsoft\\WindowsApps;C:\\Users\\E048374\\AppData\\Local\\Programs\\Microsoft VS Code\\bin;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\cmd;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\usr\\bin\\vendor_perl;C:\\Users\\E048374\\AppData\\Local\\Programs\\Git\\usr\\bin\\core_perl'

In [2]:
dsn_tns = cx_Oracle.makedsn('ctx4-scan', 
                            '1521', 
                            service_name=os.getenv("PLAN_DB_SERVICENAME")) 

In [3]:
conn = cx_Oracle.connect(user=os.getenv("PLAN_DB_USER"), 
                         password=os.getenv("PLAN_DB_PASS"), 
                         dsn=dsn_tns)

In [4]:
route = '52'
start_date = '01-NOV-19' #'DD-MON-YY' format
end_date = '30-NOV-19' #'DD-MON-YY' format

In [5]:
query = f'''with bus_dates as (
select svc_date, bus_id, count(*)
from 
(select * from 
BUS_STATE_PAX_LOAD_HMBRW_V01 
where svc_date between to_date('{start_date}','DD-MON-YY') and to_date('{end_date}','DD-MON-YY')
and route_id like '{route}%')
group by svc_date, bus_id
order by svc_date
)
select
'rawnav'||'0'||bus_id||to_char(svc_date, 'YYMMDD')||'.txt' as filename
from
bus_dates'''

In [6]:
rawnav_file_df = pd.read_sql_query(query, conn)

In [7]:
rawnav_file_df.head(3)

,FILENAME
0,rawnav03212191101.txt
1,rawnav06498191101.txt
2,rawnav06500191101.txt


In [8]:
rawnav_file_df.to_csv(os.path.join(path_processed_data, "rawnav_file_subset.csv"), index=False)

# Step 2 - while still connected to the DB - grab the schedule data
Update script below to grab the schedule of the timeframe you're interest in. Main concern is stop locations and/or stop sequence changes

In [9]:
pattern_id = '5201'
query = f'''with current_version_id as (
select max(versionid) versionid from BUS_SCHED_STOP_SEQUENCE_V)
select 
seq.route, seq.variation as pattern, seq.pattern_id, seq.directiondescription direction, seq.routename route_text
, seq.routevarname pattern_name, seq.geostopid geoid, seq.stopid stop_id, seq.directionid direction_id, route.pattern_destination
,seq.geostopdescription as geo_description, seq.latitude stop_lat, seq.longitude stop_lon, seq.stop_sequence, route.pattern_distance trip_length
from BUS_SCHED_STOP_SEQUENCE_V seq 
join current_version_id curr on (curr.versionid = seq.versionid)
left join bus_sched_route_v route on (seq.versionid = route.versionid and seq.routevarid = route.ROUTEVARID)
where seq.PATTERN_ID = '{pattern_id}' '''

In [10]:
bus_sched_df = pd.read_sql_query(query, conn)

In [11]:
bus_sched_df.head()

,ROUTE,PATTERN,PATTERN_ID,DIRECTION,ROUTE_TEXT,PATTERN_NAME,GEOID,STOP_ID,DIRECTION_ID,PATTERN_DESTINATION,GEO_DESCRIPTION,STOP_LAT,STOP_LON,STOP_SEQUENCE,TRIP_LENGTH
0,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,27665,23875,13,METROBUS,D ST SW + 7TH ST SW,38.884918,-77.020996,40,35197
1,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,21969,25811,13,METROBUS,7TH ST + I ST,38.879684,-77.021767,39,35197
2,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,21970,19046,13,METROBUS,MAINE AVE + 9TH STREET,38.879967,-77.024635,38,35197
3,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,4773,7197,13,METROBUS,12TH ST + C ST,38.885567,-77.028114,37,35197
4,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,4897,40843,13,METROBUS,INDEPENDENCE AVE + 12TH ST,38.887402,-77.028580,36,35197


In [12]:
#close connection
conn.close()

In [13]:
bus_sched_df.sort_values('STOP_SEQUENCE').iloc[12:20]

,ROUTE,PATTERN,PATTERN_ID,DIRECTION,ROUTE_TEXT,PATTERN_NAME,GEOID,STOP_ID,DIRECTION_ID,PATTERN_DESTINATION,GEO_DESCRIPTION,STOP_LAT,STOP_LON,STOP_SEQUENCE,TRIP_LENGTH
27,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,7864,22172,13,METROBUS,14TH ST + PARKWOOD PL,38.935799,-77.032791,13,35197
26,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,7714,6206,13,METROBUS,14TH ST + NEWTON ST,38.932697,-77.032784,14,35197
25,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,7593,6207,13,METROBUS,14TH ST + PARK RD,38.930527,-77.032799,15,35197
24,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,16203,21627,13,METROBUS,14TH ST + IRVING ST,38.928444,-77.032646,16,35197
23,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,16662,19066,13,METROBUS,14TH ST + HARVARD ST,38.926239,-77.032471,17,35197
22,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,7243,19143,13,METROBUS,14TH ST + FAIRMONT ST,38.924294,-77.032326,18,35197
21,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,7113,21180,13,METROBUS,14TH ST NW + CHAPIN ST NW,38.921200,-77.032013,19,35197
20,52,01,5201,SOUTH,14th Street Line,[52]14TH+COLORADO - L ENFANT PLAZA,7021,17555,13,METROBUS,14TH ST + FLORIDA AVE,38.919769,-77.032066,20,35197


In [14]:
bus_sched_df.columns= bus_sched_df.columns.str.lower()
bus_sched_df.to_csv(os.path.join(path_processed_data, f"bus_sched_{pattern_id}.csv"), index=False)